# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233242743913                   -0.50    7.0         
  2   -7.250006335755       -1.78       -1.40    1.0   5.04ms
  3   -7.251249666858       -2.91       -2.05    2.0   6.12ms
  4   -7.251230944064   +   -4.73       -2.12    1.0   5.15ms
  5   -7.251325414348       -4.02       -2.61    1.0   5.18ms
  6   -7.251337802279       -4.91       -3.17    1.0   5.24ms
  7   -7.251338701477       -6.05       -3.56    2.0   6.38ms
  8   -7.251338790244       -7.05       -4.09    1.0   69.2ms
  9   -7.251338796535       -8.20       -4.33    3.0   7.22ms
 10   -7.251338798309       -8.75       -4.77    1.0   5.48ms
 11   -7.251338798664       -9.45       -5.41    2.0   6.49ms
 12   -7.251338798696      -10.49       -5.54    3.0   7.56ms
 13   -7.251338798704      -11.13       -6.20    1.0   5.92ms
 14   -7.251338798704      -12.47       -6.20    3.0   7.93ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.037508054833337766
[ Info: Arnoldi iteration step 2: normres = 0.7462827819007537
[ Info: Arnoldi iteration step 3: normres = 0.751041321923003
[ Info: Arnoldi iteration step 4: normres = 0.3663078444042354
[ Info: Arnoldi iteration step 5: normres = 0.3355531560890057
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.74e-02, 6.62e-02, 2.39e-01, 2.21e-01, 2.35e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5047226017619406
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.41e-02, 8.63e-02, 4.64e-01, 1.27e-01, 1.25e-01)
[ Info: Arnoldi iteration step 7: normres = 0.08251684342579076
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.73e-04, 1.60e-02, 3.57e-02, 4.56e-02, 4.51e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1275814926007949
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.77e-05